In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests11 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:
def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000
adapt_c = False
data_tuned_epsilon = False
epsilon = .5

size weirdness rn...

In [5]:
a = 0.25
num_params=9

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.95 & 0.94 & 0.80 & 0.94 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.06 & 0.01 & 0.01   \\
Model 2 & 0.05 & 0.05 & 0.05 & 0.14 & 0.05 & 0.02   \\
\hline
\end{tabular}
(array([0.948, 0.004, 0.048]), array([0.948, 0.004, 0.048]), array([0.944, 0.008, 0.048]), array([0.796, 0.06 , 0.144]), array([0.944, 0.008, 0.048]), array([0.972, 0.012, 0.016]), -4.121793574188439, 10.094007454938863, 11.058629679083998)


# power 0.... need to see...

In [6]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [ ]:
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/5) )

nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/50,nobs**(1/10) )

nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

# a  = .25, k= 9

In [ ]:
a = 0.25
num_params=9



In [ ]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 500

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 100

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

# a  = .25, k= 4, something not right?

In [ ]:
a = 0.25
num_params=4



In [ ]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

# a  = .25, K = 19

In [ ]:
a = 0.25
num_params=19


In [ ]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

# evidence of power

In [ ]:
nobs = 500

a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 100

a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

# evidence of power 2

In [ ]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 500


a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 100


a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)